In [ ]:
import terrapyn as tp

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
# from terrapyn.scoring import metrics, grouped_scores, score_df, ConfusionMatrix

# from sklearn.metrics import confusion_matrix
import geopandas as gpd
import matplotlib.pyplot as plt
from odc.geo.geobox import GeoBox
from odc.geo.geom import Geometry
import  odc.geo.xr

In [ ]:
dask_client = tp.dask_utils.create_cluster_and_client()

In [ ]:
dask_client

In [ ]:
ds = xr.open_zarr(
    # 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    'gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2',

    chunks=None,
    # chunks={'time': 48},
    storage_options=dict(token='anon'),
)

In [ ]:
ds['t2m'].isel(time=-1)

In [ ]:
start_date = dt.datetime(1980, 1, 1)
end_date = dt.datetime(1980, 12, 31)
# end_date = dt.datetime(2009, 12, 31)

In [ ]:
ds_t2m = ds['2m_temperature'].sel(time=slice(start_date, end_date))

In [ ]:
ds_sub = ds_t2m.sel(latitude=43, longitude=6, method='nearest')

In [ ]:
ds_sub = ds_sub.compute()

In [ ]:
ds_sub = tp.conversion.kelvin_to_celsius(ds_sub)

In [ ]:
ds_daily = tp.time.resample_time(ds_sub, freq='1D', resample_method='mean')

In [ ]:
ds_daily.plot();

In [ ]:
geopolygon = odc.geo.geom.box(left=0, bottom=40, right=10, top=46, crs='EPSG:4326')
geobox = GeoBox.from_geopolygon(geopolygon, resolution=0.25)

In [ ]:
ds_t2m_sub = tp.space.crop(ds_t2m, geopolygon=geopolygon, lon_name='longitude', lat_name='latitude')

In [ ]:
ds_plot = ds_t2m_sub.sel(time=start_date).compute()

In [ ]:
ds_plot.plot();
# ds_plot#.isel(time=0).where(da_lsm).plot();

In [ ]:
da_lsm = ds['land_sea_mask'].isel(time=-1)

In [ ]:
# STANDARD_TIME_DIMS = ["time", "date"]
# from typing import Optional, Tuple, Union
# import xarray
# def time_dim(
#     xx: Union[xarray.DataArray, xarray.Dataset], relaxed: bool = False
# ) -> Optional[Tuple[str, str]]:
#     """
#     Find time dimensions of ``xx``.

#     Checks for presence of dimensions named: ``time | date``

#     If ``relaxed=True`` and none of the above dimension names are found,
#     assume that the last dimension is time.

#     :returns: ``None`` if no dimensions with expected names are found
#     :returns: ``'time' | 'date'``
#     """
#     _dims = [str(dim) for dim in xx.dims]
#     guess = [dim for dim in _dims if dim in STANDARD_TIME_DIMS]
#     if guess:
#         return guess[0]

#     # Assume data has at least latitude, longitude and time
#     # dimensions, where time is the last
#     if relaxed and len(_dims) >= 2:
#         return _dims[-1]

#     return None
# rasterized = odc.geo._xr_interop.rasterize(
#         poly=geopolygon,
#         how=xx.odc.geobox,
#         all_touched=True,
#     )

In [ ]:
from terrapyn import TEST_DATA_DIR
data = xr.open_dataset(TEST_DATA_DIR / "lat_10_lon_10_time_10_D_test_data.nc")

In [ ]:
data = data.expand_dims({'band':['a', 'b']}, axis=0)

In [ ]:
data = data.set_coords({'band':['a', 'b']})

In [ ]:
geopolygon = odc.geo.geom.box(left=12.3, bottom=3.3, right=14.4, top=5.7, crs='EPSG:4326')
geopolygon


In [ ]:
crs = 'EPSG:4326'

In [ ]:
data = data.odc.assign_crs(crs=crs)

In [ ]:
# xx = data.odc.crop(poly=geopolygon, apply_mask=False)
xx = data.copy()


In [ ]:
masked = odc.geo.xr.crop(xx, geopolygon, all_touched=True)

In [ ]:
masked#.sel(band='b').isel(time=5)['var'].plot.imshow();

In [ ]:
# Repeat rasterized mask across all non-spatial dims
    sdims = spatial_dims(xx, relaxed=True)
    non_sdims = {}
    non_sdims_axis = []
    for i, (k, v) in enumerate(xx.sizes.items()):
        if k not in sdims:
            non_sdims[k] = v
            non_sdims_axis.append(i)
    if non_sdims:
        rasterized = rasterized.expand_dims(non_sdims, axis=non_sdims_axis)

In [ ]:
# Get spatial dims then broadcast mask across all remaining dims
from odc.geo._xr_interop import spatial_dims, rasterize
import xarray

In [ ]:
def mask(
    xx, poly: Geometry, invert: bool = False, all_touched: bool = True
):
    """
    Apply a polygon geometry as a mask, setting all
    :py:class:`xarray.Dataset` or :py:class:`xarray.DataArray` pixels
    outside the rasterized polygon to ``NaN``.

    :param xx:
       :py:class:`~xarray.Dataset` or :py:class:`~xarray.DataArray`.

    :param poly:
       A :py:class:`odc.geo.geom.Geometry` polygon used to mask ``xx``.

    :param invert:
        Whether to invert the mask before applying it to ``xx``. If
        ``True``, only pixels inside of ``poly`` will be masked.

    :param all_touched:
        If ``True``, the rasterize step will burn in all pixels touched
        by ``poly``. If ``False``, only pixels whose centers are within
        the polygon or that are selected by Bresenham's line algorithm
        will be burned in.

    :return:
        A :py:class:`~xarray.Dataset` or :py:class:`~xarray.DataArray`
        masked by ``poly``.

    .. seealso:: :py:meth:`odc.geo.xr.rasterize`
    """
    # Rasterise `poly` into geobox of `xx`
    rasterized = rasterize(
        poly=poly,
        how=xx.odc.geobox,
        all_touched=all_touched,
        value_inside=not invert,
    )

    # Repeat rasterized mask across all non-spatial dims
    sdims = spatial_dims(xx, relaxed=True)
    non_sdims = {}
    non_sdims_axis = []
    for i, (k, v) in enumerate(xx.sizes.items()):
        if k not in sdims:
            non_sdims[k] = v
            non_sdims_axis.append(i)
    if non_sdims:
        rasterized = rasterized.expand_dims(non_sdims, axis=non_sdims_axis)

    # Mask data outside rasterized `poly`
    xx_masked = xx.where(rasterized.data)

    # Remove nodata attribute from arrays
    if isinstance(xx_masked, xarray.Dataset):
        for var in xx_masked.data_vars:
            xx_masked[var].attrs.pop("nodata", None)
    else:
        xx_masked.attrs.pop("nodata", None)

    return xx_masked

In [ ]:
sdims = spatial_dims(xx, relaxed=True)
sdims
# non_sdims = {k:v for k, v in xx.sizes.items() if k not in sdims}

non_sdims = {}
non_sdims_axis = []
for i, (k, v) in enumerate(xx.sizes.items()):
    if k not in sdims:
        non_sdims[k] = v
        non_sdims_axis.append(i)
non_sdims, non_sdims_axis

In [ ]:
invert: bool = False
all_touched: bool = True
rasterized = rasterize(
        poly=geopolygon,
        how=xx.odc.geobox,
        all_touched=all_touched,
        value_inside=not invert,
    )


In [ ]:
# repeat rasterized mask across all non-spatial dims
rasterized = rasterized.expand_dims(non_sdims, axis=non_sdims_axis)

In [ ]:
tp.space.crop(ds, geopolygon=geopolygon, )#["var"].values

In [ ]:
geopolygon = odc.geo.geom.box(left=0, bottom=0, right=25, top=4, crs='EPSG:4326')
geobox = GeoBox.from_geopolygon(geopolygon, resolution=1)

In [ ]:
import odc.geo.xr

In [ ]:
geopolygon.

In [ ]:
da = odc.geo.xr.rasterize(geopolygon, how=1).rename({'latitude': 'lat', 'longitude': 'lon'})

In [ ]:
da.values = 1

In [ ]:
da.fill

In [ ]:
 df = pd.DataFrame(
            {
                "time": pd.date_range("2019-03-15", freq="1D", periods=3),
                "id": [123, 456, 789],
                "lat": [1, 3, 5],
                "lon": [10, 20, 30],
            }
        ).set_index(["time", "id"])

In [ ]:
tp.space.crop(df, geopolygon)

In [ ]:
b = gpd.read_parquet('/Users/chill/software/regrowag/weather-analysis/data/boundaries.parquet')

In [ ]:
b

In [ ]:
da_world = tp.space.generate_grid(return_type='dataarray', resolution=1, fill_value=1)

In [ ]:
shapely_geometry = b.iloc[0].geometry

In [ ]:
geopolygon = Geometry(shapely_geometry, crs="EPSG:4326")

In [ ]:
geobox = GeoBox.from_geopolygon(geopolygon, resolution=0.01)

### crop

In [ ]:
import typing as T
import odc.geo.xr
from terrapyn.space import get_lat_lon_from_data

In [ ]:
def generate_grid(
    geobox: BBox = None,
    resolution: float = 1.0,
    min_lat: float = -90,
    max_lat: float = 90,
    min_lon: float = -180,
    max_lon: float = 180,
    return_type: str = "dataarray",
    fill_value: T.Union[int, float] = None,
) -> T.Union[T.Tuple[np.ndarray, np.ndarray], xr.DataArray, xr.Dataset]:
    """
    Makes an xr.DataArray/xr.Dataset grid with optional variable and fill value.

    Args:
        resolution: Resolution (in degrees).
        bbox: Optional Bounding box for data. Takes precedence over other coordinates.
        min_lat: Lower latitude.
        max_lat: Upper latitude.
        min_lon: Lower longitude.
        max_lon: Upper longitude.
        return_type: Type of object to return; 'numpy' for arrays of latitude, longitude, 'dataarray' for Xarray DataArray.
        fill_value: Value used to fill the grid. Defaults to `np.nan` if `fill_value` is `None`.

    Returns:
        Arrays or DataArray with the given coordinates, with optional fill value.
    """
    if bbox:
        max_lat, min_lon, min_lat, max_lon = bbox.NWSE
    lats = np.arange(min_lat, max_lat + resolution, resolution)
    lons = np.arange(min_lon, max_lon + resolution, resolution)

    if return_type == "numpy":
        return lats, lons

    if fill_value:
        data = np.full(shape=(lats.shape[0], lons.shape[0]), fill_value=fill_value)
    else:
        data = np.nan

    da = xr.DataArray(data=data, coords={"lat": lats, "lon": lons}, dims=["lat", "lon"])
    return da

In [ ]:
data = da_world.copy()

In [ ]:
df = data.to_dataframe('val').copy()

In [ ]:
gdf = data.to_dataframe('val').copy()
gdf.reset_index(inplace=True)
gdf['geometry'] = gpd.points_from_xy(gdf['lon'], gdf['lat'])
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')

In [ ]:
gdf = gdf.set_crs("EPSG:4326")
gdf = gdf.to_crs('epsg:3857')

In [ ]:
crop(data=df, geopolygon=geopolygon)

In [ ]:
crs="EPSG:4326"

In [ ]:
geopolygon.crs

In [ ]:
data.odc.assign_crs(crs=crs)

In [ ]:
if not hasattr(data, 'spatial_ref'):
    data = data.odc.assign_crs(crs=crs)
return data.odc.crop(poly=geopolygon, apply_mask=apply_mask, all_touched=all_touched)

In [ ]:
# geopolygon = Geometry(geom, crs="EPSG:4326")
# Mask data to set pixels outside polygon to NaN
# da_masked = da.odc.mask(poly=geopolygon)
# Crop data to extent of polygon (and optionally mask)


In [ ]:

da = odc.geo.xr.assign_crs(da, crs="EPSG:4326")
# geopolygon = Geometry(geom, crs="EPSG:4326")
# Mask data to set pixels outside polygon to NaN
# da_masked = da.odc.mask(poly=geopolygon)
# Crop data to extent of polygon (and optionally mask)
da_cropped = da.odc.crop(poly=geopolygon, apply_mask=True)a

In [ ]:
geobox.boundingbox

In [ ]:
bbox = tp.space.BBox(geom)

In [ ]:
min_lon, min_lat, max_lon, max_lat = geobox.boundingbox.bbox
geobox.boundingbox

In [ ]:
da = tp.space.generate_grid(max_lon=max_lon, max_lat=max_lat, min_lat=min_lat, min_lon=min_lon, 
                            return_type='dataarray', 
                            # bbox=bbox, 
                            resolution=0.1, fill_value=1)

In [ ]:
da

In [ ]:
ds = da.to_dataset(name='a')

In [ ]:
# ds['b'] = da['a'] + 1

In [ ]:
# GeoBox.from_bbox(bbox=(102.12744140625, 8.583251953125, 109.44492187500003, 23.34521484375), crs='EPSG:4326', resolution=0.1)

In [ ]:
geobox = GeoBox.from_geopolygon(geopolygon, resolution=0.01)
# geobox

In [ ]:
da = odc.geo.xr.assign_crs(da, crs="EPSG:4326")
# geopolygon = Geometry(geom, crs="EPSG:4326")
# Mask data to set pixels outside polygon to NaN
# da_masked = da.odc.mask(poly=geopolygon)
# Crop data to extent of polygon (and optionally mask)
da_cropped = da.odc.crop(poly=geopolygon, apply_mask=True)

In [ ]:
da_cropped.plot()

In [ ]:
GeoBox.from_geopolygon(geom, resolution=(0.1, 0.1))

In [ ]:
geobox = GeoBox.from_bbox(
   (-2_000_000, -5_000_000,
   2_250_000, -1_000_000),
   "epsg:3577", resolution=1000)


In [ ]:
geobox.coords

In [ ]:
lons = np.random.uniform(-180, 180, 100)
lats = np.random.uniform(-90, 90, 100)
# sine and cos of lats and lons
values = np.sin(np.pi/180 *lons) * np.cos(np.pi/180 *lats)

In [ ]:
ilats, ilons = tp.space.generate_grid(return_type='numpy')

In [ ]:
plt.scatter(x=lons, y=lats, c=values);

In [ ]:
z = tp.space.inverse_distance_weighting(lons=lons, lats=lats, values=values, lons_out=ilons, lats_out=ilats, p=2, return_type='numpy')

In [ ]:
plt.contourf(ilons, ilats, z, levels=20);

In [ ]:
np.random.seed(42)
n_lat = 4
n_lon = 4
n_time = 3
data = 5 + np.random.randn(n_time, n_lat, n_lon)
da = xr.DataArray(
    data,
    dims=["time", "lat", "lon"],
    coords={
        "time": pd.date_range("2014-09-06", periods=n_time),
        "lat": 3 + np.arange(n_lat),
        "lon": 13 + np.arange(n_lon),
    },
    name="var",
)
ds = da.to_dataset()
df = ds.to_dataframe().rename(columns={"var": "var1"})
df["var2"] = df["var1"] * 0.9
df["var3"] = df["var1"] * 0.8
df["var4"] = df["var1"] * 0.7
df["model"] = df["var1"] * 0.95


In [ ]:
tp.scoring.score_df(df, metric='me', model_names="var1", obs_names="model")

In [ ]:
metrics = ["me", "mae", "rmse"]
df_list = []
for metric in metrics:
    print(f"metric: {metric}")
    df_list.append(tp.scoring._scoring._score_df(df, metric=metric, model_names=['var2', 'var3', 'model'], obs_names=['var1', 'var4']))

In [ ]:
df_list[0]

In [ ]:
a = pd.concat(df_list, axis=1).T

In [ ]:
a

In [ ]:
a.index = metrics

In [ ]:
a

In [ ]:
# # Generate a datframe with 3 months of daily data for 3 stations
# np.random.seed(0)
# n = 90
# stations = ['a', 'b', 'c']
# dates = pd.date_range('2020-01-01', periods=n, freq='D')
# data = np.random.rand(n * len(stations), 2)
# df = pd.DataFrame(data, index=pd.MultiIndex.from_product([dates, stations], names=['date', 'id']), columns=['tmax', 'tmin'])
# df["tmin_obs"] = df["tmin"] * 0.9
# df["tmax_obs"] = df["tmax"] * 0.9
# df["qc_flag"] = np.random.rand(len(df)) > 0.5
# model_names = ["tmax", "tmin"]
# obs_names = ["tmax_obs", "tmin_obs"]
# # Convert values to binary for confusion matrix
# threshold = 0.5
# truth = df["tmax"] > threshold
# pred = df["tmax_obs"] > threshold

In [ ]:
metrics=["me", "mae", "rmse"],
grouping_keys=["time", "id", "qc_flag"],
time_grouping="month",

In [ ]:
result = grouped_scores(
    df,
    metrics=["me", "mae", "rmse"],
    groupby_time=False,
    other_grouping_keys=["id", "qc_flag"],
    model_names=model_names,
    obs_names=obs_names,
    output_index_names=model_names,
)

In [ ]:
result.loc[("a", False)].values

In [ ]:
result#[0:2, 0:2] 

In [ ]:
ds

In [ ]:

lats = [3.32, 4.67]
lons = [15.02, 15.73]
ids = ["a", "b"]

In [ ]:
[a if a is not None else b] + ['c']

In [ ]:
import terrapyn as tp
from google.cloud import bigquery
import datetime as dt
import shapely

In [ ]:
gsod = tp.bq.data.NOAA_GSOD()
gsod

In [ ]:
ghcn = tp.bq.data.NOAA_GHCN()
ghcn

In [ ]:
start_date = dt.date(2000, 1, 1)
end_date = dt.date(2023, 12, 31)
geom = shapely.Point(6, 43).buffer(0.2)
# geom = shapely.Point(106.299, 16.646).buffer(1)

In [ ]:
ghcn_stations = ghcn.stations(start_date=start_date, end_date=end_date, geom=geom)

In [ ]:
ghcn_stations

In [ ]:
gsod_stations = gsod.stations(start_date=start_date, end_date=end_date, geom=geom)

In [ ]:
gsod_stations

In [ ]:
# station_ids = ['USC00248783']

In [ ]:
# df = gsod.stations(geom=geom)

In [ ]:
# df = gsod.data(start_date=start_date, end_date=end_date, station_ids=stations['id'])
df = ghcn.data(start_date=start_date, end_date=end_date, station_ids=stations['id'])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
avg = tp.time.groupby_time(data=df, time_dim='date', grouping='year', other_grouping_keys=['id']).agg(['mean', 'std'])

In [ ]:
import pandas as pd

In [ ]:
idx = pd.IndexSlice
avg.loc[idx[:, 'FRE00171640'], 'tmax'].droplevel(1).plot.bar(y='mean', yerr='std', rot=0);

In [ ]:
import pandas as pd

In [ ]:
df.loc[df['elevation'].isna() | df['elevation'].eq(-999), 'elevation'] = 0.0

In [ ]:
df['elevation'] = df['elevation'].astype(float)

In [ ]:
df['elevation'].plot.hist(bins=100)

In [ ]:
query = f"""
    select
        item_id,
        count(*) as count
    from `{field_soil_properties_table_id}`
    group by item_id
    having count > 1
"""
query_job = client.query(query)
n_duplicates = query_job.result().total_rows
if n_duplicates > 0:
    print(f"{n_duplicates:,d} duplicates in {field_soil_properties_table_id}")
else:
    print(f"No duplicates in {field_soil_properties_table_id}")

In [ ]:
query = """
      with
            stations as (
                select
                concat(usaf, wban) as id,
                st_geogpoint(lon, lat) as geom,
                parse_numeric(elev) as elevation,
                cast(`begin` as date format 'YYYYMMDD') as start_date,
                cast(`end` as date format 'YYYYMMDD') as end_date,
                from `bigquery-public-data.noaa_gsod.stations`
                where lat != 0 and lon != 0
            )
            select * from stations
"""

bdf = bpd.read_gbq(query)

In [ ]:
bdf.head()

In [ ]:
class NOAA_GSOD():
    
    def __init__(self):
        self.client = bigquery.Client()

    def stations(self, start_date: dt.datetime = None, end_date: dt.datetime = None, geom: shapely.Geometry = None):
        """
        Get the list of weather stations from the NOAA GSOD dataset.

        Args:
            start_date: The start date for the station data.
            end_date: The end date for the station data.
            geom: A shapely geometry object to filter by.
        
        Returns:
            A Pandas DataFrame with the station metadata.
        """
    `   #


        query = """
            with
            stations as (
                select
                concat(usaf, wban) as id,
                st_geogpoint(lon, lat) as geom,
                parse_numeric(elev) as elevation,
                cast(`begin` as date format 'YYYYMMDD') as start_date,
                cast(`end` as date format 'YYYYMMDD') as end_date,
                from `bigquery-public-data.noaa_gsod.stations`
                where lat != 0 and lon != 0
            )
            select * from stations
            where start_date > date(2020, 1, 1) and end_date < date(2021, 1, 1)            """
        return self.client.query(query).to_dataframe()


In [ ]:

import numpy as np
t_base = 10
t_cutoff = 30

tx = np.array([23])
tm = np.array([12])
tp.indices.growing_degree_days(tx, tm, t_base, t_cutoff)

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

In [ ]:
import importlib
import pkgutil

In [ ]:
for mod_info in pkgutil.walk_packages(__path__, __name__ + '.'):
    mod = importlib.import_module(mod_info.name)
    try:
        names = mod.__dict__['__all__']
    except KeyError:
        names = [k for k in mod.__dict__]

In [ ]:

    globals().update({k: getattr(mod, k) for k in names})

In [ ]:
import geopandas as gpd
import pandas as pd
import json
import shapely
import fiona

fiona.supported_drivers['KML'] = 'rw'

def parse_geometry(input_string):
    input_list = json.loads(input_string)
    geom_flat = [
        (pair['lng'], pair['lat'])
        for pair in input_list
    ]
    return shapely.geometry.Polygon(geom_flat)

In [ ]:
df = pd.read_csv("~/Downloads/Geometries_Pilot_KlimRegrow_2000ha.csv", delimiter=";")
# Drop rows with missing values
df = df.dropna(how='all')
# Drop duplicated rows
df.drop_duplicates(inplace=True)
df['geometry'] = df['Geometry'].map(parse_geometry)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [ ]:
gdf = gdf[['Farm ID', 'Field ID', 'geometry']].reset_index(drop=True)

In [ ]:
gdf.to_parquet("/Users/chill/software/regrowag/representative_fields/scratch/soil_sampling/Geometries_Pilot_KlimRegrow_2000ha.parquet")

In [ ]:

gdf.to_file('Geometries_Pilot_KlimRegrow_2000ha_regrowformat_edit.kml', driver='KML') ## does retain client field and farm ids

In [ ]:
import fiona
fiona.supported_drivers['KML'] = 'rw'

In [ ]:
df = gpd.read_file("/Users/chill/software/regrowag/representative_fields/scratch/soil_sampling/Geometries_Pilot_KlimRegrow_2000ha_regrowformat_edit.kml", driver='KML')

In [ ]:
import ee

In [ ]:
ee.Initialize()

In [ ]:
img = tp.ee.data.soilgrids(return_horizon_mean=True)

In [ ]:
geojson = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"coordinates":[[[105.16302156719024,10.387603943116673],[105.16302156719024,10.353064449494639],[105.19814120323588,10.353064449494639],[105.19814120323588,10.387603943116673],[105.16302156719024,10.387603943116673]]],"type":"Polygon"}},{"type":"Feature","properties":{},"geometry":{"coordinates":[[[105.09969308522113,10.367838972296724],[105.09969308522113,10.327504016078379],[105.13480832876036,10.327504016078379],[105.13480832876036,10.367838972296724],[105.09969308522113,10.367838972296724]]],"type":"Polygon"}},{"type":"Feature","properties":{},"geometry":{"coordinates":[[[105.1662707314378,10.337887984472758],[105.1662707314378,10.30473862014739],[105.19773188932533,10.30473862014739],[105.19773188932533,10.337887984472758],[105.1662707314378,10.337887984472758]]],"type":"Polygon"}}]}

In [ ]:
fc = ee.FeatureCollection(geojson)

In [ ]:
pc = tp.ee.stats.principal_components(img, fc, scale=250)

In [ ]:
import geemap

In [ ]:
pc

In [ ]:
m = geemap.Map()
m.addLayer(remapped, {}, "remapped")
m.centerObject(fc, 15)
m

In [ ]:
remapped = tp.ee.stats.bin_and_remap(img.select('phh2o'), bins=[0,2,4,6,8,10], labels=[1,2,3,4,5,6])

In [ ]:
pct = tp.ee.stats.image_percentiles(img.select('soc'), fc, scale=250)

In [ ]:
dir(tp.stats)

In [ ]:
df = tp.io.kml_to_geodataframe("tests/data/KML_Samples.kml")

In [ ]:
df

In [ ]:
df = tp.io.kmz_to_geodataframe("/Users/chill/Downloads/KML_example.kmz")

In [ ]:
df

In [ ]:
ds = xr.Dataset(data_vars={"var": (("lat", "lon", "time"), np.ones((1, 1, 100)))}, coords={"lat": [1], "lon": [2], "time": pd.date_range("2022-01-01", periods=100)})

In [ ]:
offset = pd.Timedelta(hours=1)

In [ ]:
offset

In [ ]:
ds.resample(time="M", offset='1H').mean()

In [ ]:
> groupby_freq(ds, freq="M")
        DatasetResample, grouped over '__resample_dim__'
        4 groups with labels 2022-01-31, ..., 2022-04-30.
        >>> groupby_freq(ds['var'].to_dataframe(), freq="M").sum()  # doctest: +NORMALIZE_WHITESPACE

In [ ]:
np.random.seed(42)
n_lat = 4
n_lon = 4
n_time = 3
data = 5 + np.random.randn(n_time, n_lat, n_lon)

da = xr.DataArray(
    data,
    dims=["time", "lat", "lon"],
    coords={
        "time": pd.date_range("2014-09-06", periods=n_time),
        "lat": 3 + np.arange(n_lat),
        "lon": 13 + np.arange(n_lon),
    },
    name="var",
)
ds = da.to_dataset()

lats = [3.32, 4.67]
lons = [15.02, 15.73]
ids = ["a", "b"]


In [ ]:

def test_nearest_coordinate_values(self):
    df = tp.space.get_data_at_coords(self.ds, lats=self.lats, lons=self.lons)
    nearest_values_point_0 = df.loc[(slice(None), 0), "var"].values
    np.testing.assert_almost_equal(nearest_values_point_0, np.array([5.64768854, 4.09197592, 5.82254491]))
    nearest_values_point_1 = df.loc[(slice(None), 1), "var"].values
    np.testing.assert_almost_equal(nearest_values_point_1, np.array([4.53427025, 5.37569802, 4.6988963]))

def test_missing_latitudes(self):
    self.assertRaises(
        ValueError,
        tp.space.get_data_at_coords,
        ds=self.ds,
        lats=None,
        lons=self.lons,
    )

def test_linear_values(self):
    df = tp.space.get_data_at_coords(self.ds, lats=self.lats, lons=self.lons, method="linear")
    nearest_values_point_0 = df.loc[(slice(None), 0), "var"].values
    np.testing.assert_almost_equal(nearest_values_point_0, np.array([5.95248557, 4.38774388, 5.11628122]))
    nearest_values_point_1 = df.loc[(slice(None), 1), "var"].values
    np.testing.assert_almost_equal(nearest_values_point_1, np.array([5.01396221, 4.63833409, 4.7608919]))

def test_id_names(self):
    df = tp.space.get_data_at_coords(self.ds, lats=self.lats, lons=self.lons, point_names=self.ids, method="linear")
    self.assertEqual(
        list(df.index.get_level_values("id")),
        ["a", "b", "a", "b", "a", "b"],
    )

def test_point_names_dim(self):
    df = tp.space.get_data_at_coords(
        self.ds,
        lats=self.lats,
        lons=self.lons,
        point_names=self.ids,
        method="linear",
        point_names_dim="test",
    )
    self.assertEqual(list(df.index.names), ["time", "test"])

def test_dataset_without_time(self):
    data = self.ds.isel(time=0).drop_vars("time")
    df = tp.space.get_data_at_coords(data, lats=self.lats, lons=self.lons, method="nearest")
    self.assertEqual(df.index.name, "id")
    np.testing.assert_almost_equal(df["var"].values, np.array([5.64768854, 4.53427025]))
